### Project 3 - Group 7 ###

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
# WARNING: Do not run this cell unless you want to replace the CSV file
import time
from IPython.core.display import clear_output
import requests

API_KEY = '3b7edf89f3e5d5af2746988a93b5c39e'
headers = {'user-agent': 'machinelearning'}
url = 'https://ws.audioscrobbler.com/2.0/'

responses = []
page = 1
total_pages = 99999

while page <= 2: # set the number of pages we want to download. replace with "total_pages" to download everything.
    payload = {'method': 'chart.gettoptracks', 'api_key': API_KEY, 'format': 'json', 'page' : page}

    print("Requesting page {}/{}".format(page, total_pages)) # print some output so we can see the status
    clear_output(wait = True)

    response = requests.get(url, headers=headers, params=payload)
    if response.status_code != 200: # if we get an error, print the response and halt the loop
        print(response.text)
        break

    page = int(response.json()['tracks']['@attr']['page']) # extract pagination info
    total_pages = int(response.json()['tracks']['@attr']['totalPages'])

    responses.append(response) # append response
    time.sleep(0.1) #avoid requesting too fast. I don't know if we will actually get banned without this
    page += 1 # increment the page number

frames = [pd.DataFrame(r.json()['tracks']['track']) for r in responses]
df = pd.concat(frames)
df.drop(['duration', 'playcount', 'listeners', 'streamable', 'image', 'url', 'mbid'], axis=1, inplace=True)
df = df.reset_index(drop=True)
df.rename(columns = {'name':'track'}, inplace = True)

# clean artist column and get artist_list[]
temp = []
artist_list = []
key = 'name'
for i in df.artist:
    temp.append(i)
for d in temp:
    artist_list.append(d[key])
df.artist = artist_list

#get track_list
track_list = []
for i in df.track:
    track_list.append(i)

responses = []
counter = 0
while counter < len(track_list):
    payload = {'method': 'track.gettoptags', 'artist': artist_list[counter], 'track': track_list[counter], 'api_key': API_KEY, 'format': 'json', 'page' : page}
    print("Requesting tag {}/{}".format(counter, len(track_list))) # print some output so we can see the status
    clear_output(wait = True)
    response = requests.get(url, headers=headers, params=payload)
    if response.status_code != 200: # if we get an error, print the response and halt the loop
        print(response.text)
        break
    responses.append(response)
    time.sleep(0.1) #avoid requesting too fast. I don't know if we will actually get banned without this
    counter +=1

genres = []
for r in responses:
    genres.append(r.json()['toptags']['tag'][0]['name'])
df['genre'] = genres

# save df as csv file
df.to_csv('dataset.csv', index=False)
clear_output(wait = True)
print("CSV file written")

CSV file written


In [4]:
dataset = pd.read_csv('dataset.csv')
dataset

,track,artist,genre
0,As It Was,Harry Styles,indie pop
1,N95,Kendrick Lamar,rap
2,United In Grief,Kendrick Lamar,jazz rap
3,Worldwide Steppers,Kendrick Lamar,kodak black
4,Die Hard,Kendrick Lamar,rnb
...,...,...,...
145,King Kunta,Kendrick Lamar,Hip-Hop
146,Golden,Harry Styles,indie pop
147,Softcore,The Neighbourhood,alternative
148,Heartless,Kanye West,Kanye West
